# Greatest Common Divisor, Extended Euclidean Algorithm
In Lesson 6 you learned about the Euclidean Algorithm for computing the greatest common divisor. And in a recent lecture we learned about the Extended Euclidean Algorithm for computing the more general form of the greatest common divisor: $ax + by = \gcd(a, b)$.

The solution to this equation is a pair (a tuple) of integers (*x*, *y*).

Here is a recursive definition by parts of the Extended Euclidean Algorithm, which I will call **egcd**:

$$
\text{egcd}(a, b) =
\begin{cases}
(1, 0) & \text{if $b = 0$}, \\
(t, s - q \cdot t) & \text{otherwise, where} \\
& q = a \div b & \text{q is the integer quotient} \\
& (s, t) = \text{egcd}(b, a \bmod b)
\end{cases}
$$

Let's turn this into a Python function. I've given you some hints in the comments below.

In [2]:
def egcd(a, b):
    if b == 0:
        return (1, 0)
    else:
        # Calculate q, the quotient, using integer division
        q = a // b        
        # Calculate (s, t) by calling egcd(b, a mod b)
        s, t = egcd(b, a % b)        
        # Return (t, s - q * t)
        return (t, s - q * t)

In [3]:
# Run this cell to check your egcd function
assert egcd(5, 37) == (15, -2)
assert egcd(81, 256) == (-79, 25)

## Multiplicative Inverse
To be able to decrypt a linear cipher, we need to solve the following equation for *p*:

$$c = p \cdot m + k \quad \pmod{n}$$

After a little algebraic manipulation, we get

$$p = \frac{c - k}{m} \quad \pmod{n}$$

But dividing by *m* would result in a fraction. Instead of dividing, we can multiply by the multiplicative inverse of *m* (mod *n*). Recall that $m^{-1} (\text{mod } n)$ is the nomenclature for the multiplicative inverse of *m* mod *n*. We rewrite the equation as:

$$p = (c - k)m^{-1} \quad \pmod{n}$$

To compute the multiplicative inverse, we'll use the Extended Euclidean Algorithm! Recall that it returns the (*x*, *y*) solution to $ax + by = \gcd(a, b)$. It turns out that *x* is the multiplicative inverse of *a* mod *b*.

Therefore, if we call egcd(*m*, *n*) and get back the tuple (*x*, *y*), the multiplicative inverse is just the *x* value.

Sometimes you get back a negative value for *x*; that's okay, but a positive integer would be better. To convert it to the congruent positive modular value, just do a final *mod n*. For example, if we were trying to compute the multiplicative inverse of 5 mod 37 and we got back &ndash;22, we could just calculate &ndash;22 mod 37 and return the desired value, 15, to the caller.

To write the Python function, simply call egcd(*a*, *n*), obtain the first element of the tuple that you get back, do a final *mod n* on it and return the result.

With that working, we will be ready to decrypt the linear cipher!

## Problem 1
Write the `multinv` function. It takes two numbers, *a* and *n*, and returns the multiplicative inverse of $a \bmod n$. There are comments in the function below to help you.

In [4]:
def multinv(a, n):
    # Call egcd(a, n) and get back a pair of integers (x, y)
    x, y = egcd(a, n)    
    # Return the first element (the x value), modulo n
    return x % n if x >= 0 else (x % n + n) % n

In [5]:
# Run this cell to check your multinv function
assert multinv(18, 47) == 34
assert multinv(81, 256) == 177
assert multinv(5, 37) == 15

In [6]:
# Try it out! This should return 177
multinv(81, 256)

177

In [7]:
# Verify 81*177 mod 256. This should return 1
81 * multinv(81, 256) % 256

1

# Decrypting the Linear Cipher
The functions you'll write below are similar to the ones you wrote for Lab 4, except they decrypt instead of encrypt. You may want to open up that notebook and refer to them.

If all this seems daunting to you, relax. You have the pieces you need to write the functions; you just have to put them together. An essential aspect of problem-solving is taking inventory of what you already know and what you have available to use.

* You've learned about lists and applying functions to elements of a list using `map`.
* You know about byte strings: special arrays of integers that appear on-screen kind of like strings.
* You know that byte strings can be accessed like lists; you can loop across them. Python's `map` makes this very easy.
* You have a working function for calculating the mutiplicative inverse of an integer.

These are all the pieces you need to finish the programs. None of the functions below are more than one or two lines of code.

## Problem 2
Write a function called **decrypt** that accepts three numbers (*c*, *m*, and *k*) and returns the corresponding plaintext (*p*) value as a number. You can assume the modulus (*n*) is 256. You will need to compute the multiplicative inverse of $m \bmod 256$ to decipher *c*; use your `multinv` function to help you.

The formula for calculating the plaintext is:

$$ p = (c - k)m^{-1} \pmod{n} $$

where $m^{-1}$ is the multiplicative inverse of $m \bmod n$ and $n$ is 256.

In [18]:
def decrypt(c, m, k):
    m_inv = multinv(m, 256)
    p = (c - k) * m_inv % 256 
    return p


In [19]:
# Run this cell to check your decrypt function
assert decrypt(80, 5, 11) == 65
assert decrypt(61, 5, 11) == 10

## Problem 3
Write a function called **linearDecipher** that accepts a multiplier (*m*), a shift amount (*k*), and the *ciphertext*. It returns the decrypted byte string.

In Problem 2 you wrote a function that decrypts a single value. Use what you learned from it to help write this function. The `map` function will come in handy.

You will also need to use the lambda technique to write a helper function. This function should take in the *multiplier* and *offset* and return an anonymous function (lambda). This new function should take one input, the *character* to decyrpt, and decrypt it using the given mulitiplier, offset, and multiplicative inverse function.

In [25]:
# Import csci26 library
from csci26 import *

# Write your function definitions here
def linearDecipherPartial(m, k):
    inv = multinv(m, 256)
    return lambda x: (x - k) * inv % 256

def linearDecipher(m, k, ciphertext):
    decrypt_func = linearDecipherPartial(m, k)
    plaintext_bytes = list(map(decrypt_func, ciphertext))
    plaintext = bytes(plaintext_bytes)    
    return plaintext

In [26]:
# Run this cell to check your function
assert linearDecipher(5, 11, b"s\x04''6\xab\xaa\x18\x04EE\xf0") == b'Hello Sierra'
assert linearDecipher(13, 50, b'AB') == b'\x8bP'

In [27]:
# Try it yourself!
linearDecipher(49, 100, b'7\xa3r\x1b6\xf5\x98\xc9\x10\xf5\x98}\xa3rg\xb5')

b'Congratulations!'